### Getting top 10 Songs for an Artist
##### Testing this out so we can later apply to our data 
####  @author: Jyontika Kapoor

In [12]:
# pip install spotipy
# pip install lxml

### The below code uses the **spotipy** package to get the top tracks of an artist. Each artist has a spotify ID which we obtain using *sp.search()*. Then we get the top tracks through the *artist_top_tracks()* method. 

In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup

C:\Users\shhwa\AppData\Local\Temp\ipykernel_89172\1220656153.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
## CLIENT ID AND CLIENT SECRET
### DO NOT PUT ON GIT

client_id = '40dbf8d1b5ea471195603e2f9a57d458'
client_secret = 'd31abd8ac9714ffe94f903efd09ccd7c'
genius_access_token = 'lOqXN-Ms2yerxQ84trh3ex5sV_QehWhhQNG2BecI0n3DcCuqV2Cq-Sb9py7U7ETO'


In [5]:

def get_top_tracks(artist_name, client_id, client_secret):
    # Initialize Spotify client with your credentials
    credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=credentials)
    
    # search for the artist to get the Spotify ID through sp.search
    results = sp.search(q='artist:' + artist_name, type='artist') 
    items = results['artists']['items']
    if not items:
        return []
    
    
    artist_id = items[0]['id']
    
    top_tracks = sp.artist_top_tracks(artist_id)
    
    return [track['name'] for track in top_tracks['tracks']]


In [6]:
artist_name = 'Selena Gomez'
top_tracks = get_top_tracks(artist_name, client_id, client_secret)
top_tracks

['Calm Down (with Selena Gomez)',
 'Lose You To Love Me',
 'Single Soon',
 'Love On',
 'It Ain’t Me (with Selena Gomez)',
 'People You Know',
 'Wolves',
 'Back To You - From 13 Reasons Why – Season 2 Soundtrack',
 'Fetish (feat. Gucci Mane)',
 'Same Old Love']

#### Now we want to get the lyrics for each of the 10 tracks. We will do this using the **genius API** and the requests & bs4 library. The idea is we will search a song name on Genius through the requests library, find a match, and use bs4 to parse the HTML and extract the lyrics.

In [7]:
def get_lyrics(song_name, artist_name, genius_access_token):
    #set up the request using access token 
    base_url = "https://api.genius.com"
    headers = {'Authorization': 'Bearer ' + genius_access_token}
    search_url = base_url + "/search" #getting ready to search!
    data = {'q': song_name} #we want to search the song name!

    response = requests.get(search_url, params=data, headers=headers)
    json = response.json()
    path = None

    # loops through the search results returned by the Genius API to find a specific song that matches name of song and artist name

    for hit in json["response"]["hits"]:
        if artist_name.lower() in hit["result"]["primary_artist"]["name"].lower():
            path = hit["result"]["path"]
            break #assuming the first is right 

    if path:
        lyrics_url = "https://genius.com" + path
        lyrics_response = requests.get(lyrics_url)
        soup = BeautifulSoup(lyrics_response.text, 'lxml')
        
        # used "inspect" element to find where lyrics are in HTML 
        lyrics_div = soup.find('div', {'data-lyrics-container': 'true'})
        
        # extract the text ... if lyrics are not found return lyrics not found
        lyrics = lyrics_div.get_text(separator='\n', strip=True) if lyrics_div else "Lyrics not found"
        return lyrics
    
    
    else:
        return "Lyrics not found"

In [8]:
artist_name = 'Kesha'
tracks = get_top_tracks(artist_name, client_id, client_secret)
tracks

['Timber',
 'TiK ToK',
 'Right Round',
 'Die Young',
 'We R Who We R',
 'Your Love Is My Drug',
 'Good Old Days',
 'Backstabber',
 'Blow',
 'Take It Off']

In [9]:
lyrics = get_lyrics(tracks[1], artist_name, genius_access_token)
print(lyrics)

[Verse 1]
Wake up in the morning feelin' like P. Diddy
(
Hey, what up, girl?
)
Grab my glasses, I'm out the door, I'm gonna hit this city
(
Let's go
)
Before I leave, brush my teeth with a bottle of Jack
'Cause when I leave for the night, I ain't coming back
[Pre-Chorus]
I'm talkin' pedicure on our toes, toes, tryin' on all our clothes, clothes
Boys blowin' up our phones, phones
Drop-toppin', playin' our favorite CDs, pullin' up to the parties
Tryna get a little bit tipsy
[Chorus]
Don't stop, make it pop, DJ, blow my speakers up
Tonight, I'ma fight 'til we see the sunlight
Tick tock on the clock, but the party don't stop, no
Oh, woah, woah, oh, oh, woah, woah, oh
Don't stop, make it pop, DJ, blow my speakers up
Tonight, I'ma fight 'til we see the sunlight
Tick tock on the clock, but the party don't stop, no
Oh, woah, woah, oh, oh, woah, woah, oh
[Verse 2]
Ain't got a care in the world, but got plenty of beer
Ain't got no money in my pocket, but I'm already here
And now the dudes are li

### Now we will retrive lyrics from the identified viral songs

In [31]:
cwd = os.getcwd()
viral_songs = f'{cwd}/data-analysis/viral.csv'
viral_songs = pd.read_csv(viral_songs)
viral_songs

,artists,songs
0,Kinfolk Thugs,Dumptruck
1,Cordelia,Little Life
2,"¥$, Kanye West, Ty Dolla $ign & Rich The Kid",CARNIVAL (feat. Playboi Carti)
3,Ariana Grande,"yes, and? (Mixed)"
4,Bobby Caldwell,What You Won't Do for Love
5,R.m.y,How I Love Being a Woman 6
6,Rihanna,Bitch Better Have My Money
7,"The Weeknd, JENNIE & Lily Rose Depp",One Of The Girls
8,Gold-Tiger,Funny
9,Beyoncé,Countdown


In [39]:
from collections import Counter

singers = []
artists = Counter(viral_songs['artists'])
for artist in artists: 
    singers.append(artist)

singers

['Kinfolk Thugs',
 'Cordelia',
 '¥$, Kanye West, Ty Dolla $ign & Rich The Kid',
 'Ariana Grande',
 'Bobby Caldwell',
 'R.m.y',
 'Rihanna',
 'The Weeknd, JENNIE & Lily Rose Depp',
 'Gold-Tiger',
 'Beyoncé',
 'Masego & FKJ',
 'Lana Del Rey',
 'Peso Pluma & Anitta',
 '21 Savage',
 'Flo Milli',
 'Muni Long',
 'ผดุง ทรงแสง (แจ๊ส)',
 'Lilithzplug']